In [7]:
from typing import List
from dataclasses import fields
import modelspec
from modelspec import field, instance_of, optional
from modelspec.base_types import Base
from typing import List
import xmltodict
import xml.etree.ElementTree as ET

In [8]:
@modelspec.define
class Population(Base):
    """
    Some description...

    Args:
        id: The id of the population
        component: the component to use in the population
        size: the size of the population
    """

    id: str = field(validator=instance_of(str))
    component: str = field(default=None, validator=optional(instance_of(str)))
    size: int = field(default=None, validator=optional(instance_of(int)))


@modelspec.define
class ExplicitInput(Base):
    """
    Some description...

    Args:
        target: the target of the input
        input: the input, e.g. pulseGenerator
    """

    target: str = field(default=None, validator=optional(instance_of(str)))
    input: str = field(default=None, validator=optional(instance_of(str)))


@modelspec.define
class Network(Base):
    """
    Some description...

    Args:
        id: The id of the network
        populations: the pops in the net
    """

    id: str = field(validator=instance_of(str))

    populations: List[Population] = field(factory=list)
    explicitInputs: List[ExplicitInput] = field(factory=list)


@modelspec.define
class PulseGenerator(Base):
    """
    Some description...

    Args:
        id: The id of the pulseGenerator
        delay: the delay
        duration: the duration
        amplitude: the amplitude
    """

    id: str = field(validator=instance_of(str))
    delay: str = field(validator=instance_of(str))
    duration: str = field(validator=instance_of(str))
    amplitude: str = field(validator=instance_of(str))


@modelspec.define
class Izhikevich2007Cell(Base):
    """
    Some description...

    Args:
        id: The id of the cell...
    """

    id: str = field(validator=instance_of(str))

    C: str = field(validator=instance_of(str))
    v0: str = field(validator=instance_of(str))
    k: str = field(validator=instance_of(str))
    vr: str = field(validator=instance_of(str))
    vt: str = field(validator=instance_of(str))
    vpeak: str = field(validator=instance_of(str))
    a: str = field(validator=instance_of(str))
    b: str = field(validator=instance_of(str))
    c: str = field(validator=instance_of(str))
    d: str = field(validator=instance_of(str))


@modelspec.define
class NeuroML(Base):
    """
    Some description...

    Args:
        id: The id of the NeuroML 2 document
        version: NeuroML version used
        networks: The networks present
    """

    id: str = field(validator=instance_of(str))
    version: str = field(validator=instance_of(str))

    izhikevich2007Cells: List[Izhikevich2007Cell] = field(factory=list)
    pulseGenerators: List[PulseGenerator] = field(factory=list)
    networks: List[Network] = field(factory=list)


nml_doc = NeuroML(id="TestNeuroML", version="NeuroML_v2.3")

izh = Izhikevich2007Cell(
        id="izh2007RS0",
        C="100pF",
        v0="-60mV",
        k="0.7nS_per_mV",
        vr="-60mV",
        vt="-40mV",
        vpeak="35mV",
        a="0.03per_ms",
        b="-2nS",
        c="-50.0mV",
        d="100pA",
)
nml_doc.izhikevich2007Cells.append(izh)

pg = PulseGenerator(
        id="pulseGen_0", delay="100ms", duration="800ms", amplitude="0.07 nA"
)
nml_doc.pulseGenerators.append(pg)

net = Network(id="IzNet")
nml_doc.networks.append(net)

net.populations.append(Population("IzhPop0", component="izh2007RS0", size=1))
net.explicitInputs.append(ExplicitInput(target="IzhPop0[0]", input="pulseGen_0"))

In [9]:
# This creates a xml element in bite format
xml_string = nml_doc.to_xml()
print(xml_string)

<?xml version="1.0" ?>
<NeuroML id="TestNeuroML" version="NeuroML_v2.3">
    <Izhikevich2007Cell id="izh2007RS0" C="100pF" v0="-60mV" k="0.7nS_per_mV" vr="-60mV" vt="-40mV" vpeak="35mV" a="0.03per_ms" b="-2nS" c="-50.0mV" d="100pA"/>
    <PulseGenerator id="pulseGen_0" delay="100ms" duration="800ms" amplitude="0.07 nA"/>
    <Network id="IzNet">
        <Population id="IzhPop0" component="izh2007RS0" size="1"/>
        <ExplicitInput target="IzhPop0[0]" input="pulseGen_0"/>
    </Network>
</NeuroML>



In [30]:
# This is the function that converts the xml element into a dictionary
def element_to_dict(element):
    """
    This convert an ElementTree element to a dictionary.

    Args:
        element: The ElementTree element to convert.

    Returns:
        The converted dictionary.
    """
    result = {}
    attrs = element.attrib
    if attrs:
        result.update(attrs)

    for child_element in element:
        child_key = child_element.tag
        child_value = element_to_dict(child_element)

        if child_key in result:
            if not isinstance(result[child_key], list):
                result[child_key] = [result[child_key]]
            result[child_key].append(child_value)
        else:
            result[child_key] = child_value

    return result

# This removes every instance of "id" from the returned dictionary
def handle_id(dictionary):
    if isinstance(dictionary, dict):
        if "id" in dictionary:
            nested_dict = {dictionary["id"]: dictionary.copy()}
            del nested_dict[dictionary["id"]]["id"]
            return {k: handle_id(v) for k, v in nested_dict.items()}
        else:
            return {k: handle_id(v) for k, v in dictionary.items()}
    elif isinstance(dictionary, list):
        return [handle_id(item) for item in dictionary]
    else:
        return dictionary

# This enable conversion into a dictionary
xml_elem = ET.fromstring(xml_string)

# The xml element is converted into a dictionary  
xml_dict = element_to_dict(xml_elem)

# Removes every occurance of "id" from the dictionary and replaces them with the key value
perfect_dict = handle_id(xml_dict)
perfect_dict

{'TestNeuroML': {'version': 'NeuroML_v2.3',
  'Izhikevich2007Cell': {'izh2007RS0': {'C': '100pF',
    'v0': '-60mV',
    'k': '0.7nS_per_mV',
    'vr': '-60mV',
    'vt': '-40mV',
    'vpeak': '35mV',
    'a': '0.03per_ms',
    'b': '-2nS',
    'c': '-50.0mV',
    'd': '100pA'}},
  'PulseGenerator': {'pulseGen_0': {'delay': '100ms',
    'duration': '800ms',
    'amplitude': '0.07 nA'}},
  'Network': {'IzNet': {'Population': {'IzhPop0': {'component': 'izh2007RS0',
      'size': '1'}},
    'ExplicitInput': {'target': 'IzhPop0[0]', 'input': 'pulseGen_0'}}}}}

In [20]:
baseline_dict = nml_doc.to_dict()
baseline_dict

{'TestNeuroML': {'version': 'NeuroML_v2.3',
  'izhikevich2007Cells': {'izh2007RS0': {'C': '100pF',
    'v0': '-60mV',
    'k': '0.7nS_per_mV',
    'vr': '-60mV',
    'vt': '-40mV',
    'vpeak': '35mV',
    'a': '0.03per_ms',
    'b': '-2nS',
    'c': '-50.0mV',
    'd': '100pA'}},
  'pulseGenerators': {'pulseGen_0': {'delay': '100ms',
    'duration': '800ms',
    'amplitude': '0.07 nA'}},
  'networks': {'IzNet': {'populations': {'IzhPop0': {'component': 'izh2007RS0',
      'size': 1}},
    'explicitInputs': [{'target': 'IzhPop0[0]', 'input': 'pulseGen_0'}]}}}}

In [35]:
# Checking baseline dictionary with xml generated dictionary o check equality 
def check(reference_data, check_data):
    if reference_data == check_data:
        print("True")
    else:
        print("False")

# Ckecked baseline dict with element_to_dict function and nml_dict         
check(baseline_dict, perfect_dict)

False


In [32]:
# Using another library as a baseline for checking
comparison_dict = xmltodict.parse(xml_string)

# Removes all instances of "@" used to represent attributes in the xml to dict library
def remove_at_symbol(dictionary):
    if isinstance(dictionary, dict):
        new_dict = {}
        for key, value in dictionary.items():
            new_key = key.lstrip('@')
            new_dict[new_key] = remove_at_symbol(value)
        return new_dict
    elif isinstance(dictionary, list):
        return [remove_at_symbol(item) for item in dictionary]
    else:
        return dictionary

at_removed = remove_at_symbol(comparison_dict)
handle_id(at_removed)

{'NeuroML': {'TestNeuroML': {'version': 'NeuroML_v2.3',
   'Izhikevich2007Cell': {'izh2007RS0': {'C': '100pF',
     'v0': '-60mV',
     'k': '0.7nS_per_mV',
     'vr': '-60mV',
     'vt': '-40mV',
     'vpeak': '35mV',
     'a': '0.03per_ms',
     'b': '-2nS',
     'c': '-50.0mV',
     'd': '100pA'}},
   'PulseGenerator': {'pulseGen_0': {'delay': '100ms',
     'duration': '800ms',
     'amplitude': '0.07 nA'}},
   'Network': {'IzNet': {'Population': {'IzhPop0': {'component': 'izh2007RS0',
       'size': '1'}},
     'ExplicitInput': {'target': 'IzhPop0[0]', 'input': 'pulseGen_0'}}}}}}

In [36]:
# Ckecked baseline dict with the xmltodict library function and nml_dict
check(baseline_dict, comparison_dict)

False
